In [14]:
from dotenv import load_dotenv
import streamlit as st
from langchain_community.utilities import SQLDatabase
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from sqlalchemy import URL
from IPython.display import display, Markdown

load_dotenv()

True

In [2]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [3]:
def connect_db(host, username, password, database):
    db_url = URL.create(
        host=host,
        username=username,
        password=password,
        database=database,
        drivername="postgresql",
    )
    return SQLDatabase.from_uri(db_url)

db = connect_db(
    host="ep-cool-leaf-a1sn945g-pooler.ap-southeast-1.aws.neon.tech",
    username="neondb_owner",
    password="npg_oYRTCrUmf75l",
    database="neondb",
)

In [4]:
prompt_template = ChatPromptTemplate.from_template(
        """ 
            You are an intelligent agent designed to interact with a **Postgres** SQL database. Your task is to generate accurate and syntactically correct SQL queries based on the user's input question.

            ### Responsibilities:
            - Analyze the user's question and generate a suitable SQL query.
            - Refer to the provided schema `{schema}` to understand the structure of the database.
            - Consider the chat history `{chat_history}` for context from prior messages.
            - Use only the tools provided to you to interact with the database.
            - Construct your final answer solely based on the results returned by these tools.

            ### Guidelines:
            - **Do not query all columns** (`SELECT *`). Only include the columns relevant to the user's question.
            - You may order the results by a meaningful column to return the most relevant or interesting data.
            - Always **double-check** your query before executing it.
            - If a query fails, **revise and retry** with a corrected version.
            - If SQL inbuilt funtions need to be used, remember, that should be compatible with postgres

            ### Output Rules:
            - Return **only the SQL query** with no explanation or formatting.
            - **Do not use words like "SQL" or code formatting** (e.g., `sql`, triple backticks) even for complex queries.

            ---

            **I want to know** `{question}`

        """
    )


sql_chain = RunnablePassthrough.assign(schema=lambda _: db.get_table_info()) | prompt_template| model | StrOutputParser()


In [5]:
prompt_template = ChatPromptTemplate.from_template(
            """
                You are an AI assistant who understands user's questions.
                Based upon the user's question, database schema, sql query, and sql query response after execution, 
                you will return a response in human understandable english language.

                If the response has multiple values, try to restructure in tabular format
                If the response has single value, add proper answer to understand what the value means w.r.t to the user question
                and so on. Try to make the response formatted.

                Here are the details:\n
                User Question: {question}\n
                Database Scema: {schema}\n
                Sql query: {query}\n
                Sql query response: {query_response}\n
                Conversation_hisotry: {chat_history}
            """
    )


# chain = (
#     RunnablePassthrough.assign(query=sql_chain).assign(
#         schema=lambda _: db.get_table_info(),
#         query_response=lambda x: print(f"Executable query: {x['query']}")
#         or db.run(x["query"].replace("```sql", "").replace("```", "")),
#     )
#     | prompt_template
#     | model
#     | StrOutputParser()
# )

chain = (
    RunnablePassthrough.assign(query=sql_chain).assign(
        schema=lambda _: db.get_table_info(),
        query_response=lambda x: print(f"Executable query: {x['query']}")
        or db.run(x["query"].replace("```sql", "").replace("```", "")),
    )
    | prompt_template
    | model
    | StrOutputParser()
)

In [6]:
chat_history = []


In [7]:
user_question = "What is the total number of orders?"
chat_history.append(HumanMessage(content=user_question))

In [11]:
user_question = "Give me the customer names whi have bought Smartphone on the month of June?"
chat_history.append(HumanMessage(content=user_question))

In [15]:
response = chain.invoke({"question": user_question, "chat_history": chat_history})
chat_history.append(AIMessage(content=response))
display(Markdown((response)))

Executable query: ```sql
SELECT
  T1.customer_name
FROM customer AS T1
INNER JOIN sales AS T2
  ON T1.customer_id = T2.customer_id
INNER JOIN product AS T3
  ON T2.product_id = T3.product_id
WHERE
  T3.product_category = 'Smartphone' AND EXTRACT(MONTH FROM T2.sale_date) = 6;
```


Here are the customers who bought a Smartphone in the month of June:

| Customer Name  |
|----------------|
| Amandeep Singh |
| Prashant Kumar |
| Mainak Sinha   |

In [23]:
from langchain_core.prompts import PromptTemplate


prompt_template = PromptTemplate.from_template("Answer this: {query}")

chain1 = (
    RunnablePassthrough()
    .assign(a=lambda x: print(f"Step1: {x}"))
    .assign(query=lambda x: "what is 2 + 2 ?")
    .assign(b=lambda x: print(f"Step 2: {x}"))
    | prompt_template
    | model
    | StrOutputParser()
)
print(chain1.invoke({"query": "What is 3 + 4 ?"}))

Step1: {'query': 'What is 3 + 4 ?'}
Step 2: {'query': 'what is 2 + 2 ?', 'a': None}
2 + 2 = 4
